In [2]:
from sqlalchemy import create_engine

import fuzzywuzzy as fw
from fuzzywuzzy import process

import pandas as pd


# Importando y analizando CSV

In [3]:
#Importando el dataset

df_datos = pd.read_csv("E:\\DATASETS\Base_GPX.csv", sep = ",")
df_datos.head(15)

,Grand Prix,Date,Winner,Winner Abbreviation,Car,Laps,Time
0,Great Britain,13 May 1950,Nino Farina,FAR,Alfa Romeo,70.0,2:13:23.600
1,Monaco,21 May 1950,Juan Manuel Fangio,FAN,Alfa Romeo,100.0,3:13:18.700
2,Indianapolis 500,30 May 1950,Johnnie Parsons,PAR,Kurtis Kraft Offenhauser,138.0,2:46:55.970
3,Switzerland,04 Jun 1950,Nino Farina,FAR,Alfa Romeo,42.0,2:02:53.700
4,Belgium,18 Jun 1950,Juan Manuel Fangio,FAN,Alfa Romeo,35.0,NaN
5,France,02 Jul 1950,Juan Manuel Fangio,FAN,Alfa Romeo,64.0,2:57:52.800
6,Italy,03 Sep 1950,Nino Farina,FAR,Alfa Romeo,80.0,2:51:17.400
7,Switzerland,27 May 1951,Juan Manuel Fangio,FAN,Alfa Romeo,42.0,2:07:53.640
8,Indianapolis 500,30 May 1951,Lee Wallard,WAL,Kurtis Kraft Offenhauser,200.0,3:57:38.050
9,Belgium,17 Jun 1951,Nino Farina,FAR,Alfa Romeo,36.0,2:45:46.200


In [4]:
#Verificando la cantidad de nulos existentes en nuestro dataset cargado

total_nulos = df_datos.isnull().sum()

print(total_nulos)

Grand Prix             0
Date                   0
Winner                 0
Winner Abbreviation    0
Car                    0
Laps                   3
Time                   6
dtype: int64


# Eliminando colummnas

In [5]:
#Eliminando columnas innecesarias
df_datos.drop(['Winner Abbreviation','Car'],axis=1,inplace=True)
df_datos.head()

,Grand Prix,Date,Winner,Laps,Time
0,Great Britain,13 May 1950,Nino Farina,70.0,2:13:23.600
1,Monaco,21 May 1950,Juan Manuel Fangio,100.0,3:13:18.700
2,Indianapolis 500,30 May 1950,Johnnie Parsons,138.0,2:46:55.970
3,Switzerland,04 Jun 1950,Nino Farina,42.0,2:02:53.700
4,Belgium,18 Jun 1950,Juan Manuel Fangio,35.0,NaN


# Uso de la libreria fuzzywuzzy

Verificando si existen valores con errores tipograficos (inconsistencias)

Por ejemplo, puede existir el caso de encontrar estos valores: 


  ->  Roy Rios Zavaleta , roy rios zavaleta o  RoyRiosZavaleta
  
  Estos 3 valores hacen referencia a la misma persona, se tiene que corregir
        
Para ello:
1. Se observan los valores unicos de las columnas a observar
2. De existir valores que pueden ser iguales pero tiene algun tipografico
   que los distingue, se corrigue con ayuda de la libreria fuzzywuzzy
   
La libreria **fuzzywuzzy** posee una funcion que compara dos cadenas y retorna un valor de 1 al 100

Mientras más parecido es una cadena a la otra, el valor se acerca a 100

In [6]:
#Creando df winners para observar los valores unicos de mi columna 'Winner'
winners=df_datos['Winner'].unique()
winners.sort()
winners

array(['Alain Prost', 'Alan Jones', 'Alberto Ascari',
       'Alessandro Nannini', 'Ayrton Senna', 'Bill Vukovich',
       'Bob Sweikert', 'Bruce McLaren', 'Carlos Pace', 'Carlos Reutemann',
       'Charles Leclerc', 'Clay Regazzoni', 'Damon Hill', 'Dan Gurney',
       'Daniel Ricciardo', 'David Coulthard', 'Denny Hulme',
       'Didier Pironi', 'Eddie Irvine', 'Elio de Angelis',
       'Emerson Fittipaldi', 'Esteban Ocon', 'Felipe Massa',
       'Fernando Alonso', 'Francois Cevert', 'Gerhard Berger',
       'Giancarlo Baghetti', 'Giancarlo Fisichella', 'Gilles Villeneuve',
       'Graham Hill', 'Gunnar Nilsson', 'Heikki Kovalainen',
       'Heinz-Harald Frentzen', 'Innes Ireland', 'Jack Brabham',
       'Jackie Stewart', 'Jacky Ickx', 'Jacques Laffite',
       'Jacques Villeneuve', 'James Hunt', 'Jarno Trulli', 'Jean Alesi',
       'Jean-Pierre Beltoise', 'Jean-Pierre Jabouille', 'Jenson Button',
       'Jim Clark', 'Jim Rathmann', 'Jimmy Bryan', 'Jo Bonnier',
       'Jo Siffert', 'Jo

Poniendonos en el supuesto caso que existan inconsistencias, creariamos una funcion para analizar las
coincidencias y reemplazarias con un mismo valor para todos

La siguiente funcion recibe un dataframe, una columna en donde se buscaran todos los valores unicos
y la cadena con la cual vas a comparar con los valores unicos obtenidos

In [7]:
"""
  funcion 'reemplazar_valores'
  
  Esta funcion va a recibir un dataframe, el nombre de columna y una cadena para poder hacer las
  comparaciones y posteriormente el reemplazo de existir valores que hacen referencia a un mismo dato
  como se explico anteriormente.
  
  Args:
      dataframe: Dataframe
      columna: String
      cadena: String

  Out:
     Dataframe modificado

""" 
def reemplazar_valores(dataframe, columna, cadena):
    
    valores_unicos = dataframe[columna].unique()
    
    #scorer=fw.fuzz.token_sort_ratio: indica que el tipo métrica de puntuación sera "token_sort_ratio"
    matches = fw.process.extract(cadena, valores_unicos, limit=10, scorer=fw.fuzz.token_sort_ratio)
    
    #Obtenemos los 10 valores más con un ratio >= 45
    valores_cercanos = [matches[0] for matches in matches if matches[1] >= 45]
    
    # Obtenemos las filas con todos los valores más cercanos
    filas = dataframe[columna].isin(valores_cercanos)

    # Reemplazamos todos los valores cercanos con la cadena de entrada 
    dataframe.loc[filas, columna] = cadena
    
    print("Reemplazo hecho.")
    

In [8]:
#Llamando a la funcion
reemplazar_valores(df_datos,'Winner','Alain Prost')

Reemplazo hecho.


# Identificación de valores atípicos por medio de rango intercuartil

El rango intercuartil (IQR, por sus siglas en inglés, Interquartile Range) es una medida estadística utilizada comúnmente en la limpieza de datos para identificar y tratar valores atípicos o extremos en un conjunto de datos.

El rango intercuartil se basa en encontrar 2 limites: **Superior e inferior**

Donde: Si se encuentran datos por encima del límite superior o por debajo del límite inferior se considera como **atípico**

Para el presente caso, hemos decidido realizar la identificación de posibles valores atípicos en la columna 'Laps'

In [9]:
# Aplicando percentiles a los datos

# Calcular el rango intercuartil (IQR) de los datos
Q1 = df_datos['Laps'].quantile(0.25)
Q3 = df_datos['Laps'].quantile(0.75)
IQR = Q3 - Q1

# Calcular los límites para identificar valores atípicos
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

print(f"Limite superior: {limite_superior}, Limite inferior: {limite_inferior}")

valores_atipicos = df_datos[(df_datos['Laps'] < limite_inferior) | (df_datos['Laps'] > limite_superior)]

valores_atipicos.count()

Limite superior: 101.5, Limite inferior: 25.5


Grand Prix    50
Date          50
Winner        50
Laps          50
Time          50
dtype: int64

Hemos hallado 50 datos posibles de valores atipicos dentro de los datos de la columna 'Laps', pero hemos decidido que se mantendran debido a que para el presente problema estos datos podrían ser valiosos aun siendo valores atípicos dentro de nuestro conjunto de datos.

# Formatear fechas

In [10]:
#Cambiando formato de las fechas

df_datos['Date'] = pd.to_datetime(df_datos['Date'],format='%d %b %Y')
df_datos.head()

,Grand Prix,Date,Winner,Laps,Time
0,Great Britain,1950-05-13,Alain Prost,70.0,2:13:23.600
1,Monaco,1950-05-21,Juan Manuel Fangio,100.0,3:13:18.700
2,Indianapolis 500,1950-05-30,Alain Prost,138.0,2:46:55.970
3,Switzerland,1950-06-04,Alain Prost,42.0,2:02:53.700
4,Belgium,1950-06-18,Juan Manuel Fangio,35.0,NaN


# Análisis y decisión sobre celdas con nulos

Existen algunas celdas con valores nulos, eliminaremos aquellas filas en donde posean un valor nulo

**Excepcion**: Exceptuaremos eliminar las filas con valores nulos en la columna 'Time' porque consideramos importante mantener el resto de valores de las otras columnas.

In [11]:
#Elimando filas donde valor de 'Laps' sea nulo
df_datos = df_datos.dropna(subset=['Laps'])
df_datos.head()

,Grand Prix,Date,Winner,Laps,Time
0,Great Britain,1950-05-13,Alain Prost,70.0,2:13:23.600
1,Monaco,1950-05-21,Juan Manuel Fangio,100.0,3:13:18.700
2,Indianapolis 500,1950-05-30,Alain Prost,138.0,2:46:55.970
3,Switzerland,1950-06-04,Alain Prost,42.0,2:02:53.700
4,Belgium,1950-06-18,Juan Manuel Fangio,35.0,NaN


# Eliminación de filas solo nulas y duplicadas

In [12]:
#Eliminando los valores que solo tengan nulos en todas las columnas
df_datos.dropna(how='all')
df_datos.head()

#Eliminando datos duplicados
df_datos = df_datos.drop_duplicates()

#Verificar si existen filas duplicadas
conteo_duplicados = df_datos.groupby(df_datos.columns.tolist()).size().reset_index(name='conteo')
conteo_duplicados[conteo_duplicados['conteo'] == 1]


,Grand Prix,Date,Winner,Laps,Time,conteo
0,70th Anniversary,2020-08-09,Max Verstappen,52.0,1:19:41.993,1
1,Abu Dhabi,2009-11-01,Sebastian Vettel,55.0,1:34:03.414,1
2,Abu Dhabi,2010-11-14,Sebastian Vettel,55.0,1:39:36.837,1
3,Abu Dhabi,2011-11-13,Lewis Hamilton,55.0,1:37:11.886,1
4,Abu Dhabi,2012-11-04,Kimi Räikkönen,55.0,1:45:58.667,1
...,...,...,...,...,...,...
1051,United States,2016-10-23,Lewis Hamilton,56.0,1:38:12.618,1
1052,United States,2017-10-22,Lewis Hamilton,56.0,1:33:50.991,1
1053,United States,2018-10-21,Kimi Räikkönen,56.0,1:34:18.643,1
1054,United States,2019-11-03,Valtteri Bottas,56.0,1:33:55.653,1


# Corrigiendo inconsistencias

In [13]:
# Existen valores que no poseen horas, asi que se corrigue para que todo tenga un mismo formato
df_datos['Time'] = df_datos['Time'].astype(str)
df_datos['Time'] = df_datos['Time'].apply(lambda x: '0:' + x if x.count(':') == 1 else x)


In [14]:
# Exportando a un csv los datos limpios
df_datos.to_csv("E:\\DATASETS\carreras.csv",index=False)

# Conexión y carga de los datos en una nueva BD

In [15]:
#Importación de datos limpios
df_limpios = pd.read_csv("E:\\DATASETS\carreras.csv")
df_limpios.head()

,Grand Prix,Date,Winner,Laps,Time
0,Great Britain,1950-05-13,Alain Prost,70.0,2:13:23.600
1,Monaco,1950-05-21,Juan Manuel Fangio,100.0,3:13:18.700
2,Indianapolis 500,1950-05-30,Alain Prost,138.0,2:46:55.970
3,Switzerland,1950-06-04,Alain Prost,42.0,2:02:53.700
4,Belgium,1950-06-18,Juan Manuel Fangio,35.0,NaN


In [16]:
import pyodbc

#CREDENCIALES DE INICIO DE SESION
server_name = 'LAPTOP-VD87HTVC\SQLEXPRESS'
nombre_bd = 'master'
db_final = 'PYTHON_FINAL'

try:
    conexion = pyodbc.connect('DRIVER={SQL Server};SERVER=' +
                              server_name + ';DATABASE=' + nombre_bd + ';Trusted_Connection=yes;')
except Exception as e:
    print("Ocurrió un error al conectar a SQL Server: ", e)

# Crear consultas de creacion de base de datos y tablas

eliminar_db = f'DROP DATABASE IF EXISTS {db_final};'
crear_db = f'CREATE DATABASE {db_final};'

esquema = f"""
            USE {db_final};
            CREATE TABLE CARRERAS (
                id INT PRIMARY KEY IDENTITY(1, 1),
                [Grand Prix] VARCHAR(50) NOT NULL,
                [Date] DATE NOT NULL,
                [Winner] VARCHAR(100) NOT NULL,
                [Laps] INT NOT NULL, 
                [Time] VARCHAR(20) NULL
            );
        """
conexion.autocommit = True
cursor = conexion.cursor()

try:
    cursor.execute(eliminar_db)
    cursor.execute(crear_db)
    cursor.execute(esquema)
    cursor.close()
except Exception as e:
    print(f"Algo ocurrio al intentar ejecutar las consultas {e}")
finally:
    conexion.close()

In [17]:
# Insertando los datos de mi nuevo datasets a mi nueva BD

from sqlalchemy import create_engine

conn_python = f'mssql+pyodbc://{server_name}/{db_final}?driver=SQL+Server&trusted_connection=yes'
engine_master = create_engine(conn_python)

try:
    df_limpios.to_sql(name='CARRERAS', con=engine_master, index=False, if_exists='replace')
    
    query = "SELECT * FROM CARRERAS"
    df_bd_python = pd.read_sql(query,engine_master)
except Exception as e:
    print("Ha ocurrido un error al crear la tabla o cargar los datos: " + str(e))
finally:
        engine_master.dispose()


# Consultando los datos a mi nueva BD

In [18]:
# Verificando la carga de los datos
df_bd_python.head()

,Grand Prix,Date,Winner,Laps,Time
0,Great Britain,1950-05-13,Alain Prost,70.0,2:13:23.600
1,Monaco,1950-05-21,Juan Manuel Fangio,100.0,3:13:18.700
2,Indianapolis 500,1950-05-30,Alain Prost,138.0,2:46:55.970
3,Switzerland,1950-06-04,Alain Prost,42.0,2:02:53.700
4,Belgium,1950-06-18,Juan Manuel Fangio,35.0,None


In [19]:
# 1° CONSULTA: Por número de vueltas, mostrar año, ganador y tiempo más rapido

df_con_1 = df_bd_python.sort_values(by=['Laps', 'Time'])

# Obtiene la primera fila de cada grupo de 'Laps' y reinicia los indices
df_con_1 = df_con_1.groupby('Laps').head(1).reset_index(drop=True)


In [20]:
df_con_1.head(20)

,Grand Prix,Date,Winner,Laps,Time
0,Belgium,2021-08-29,Max Verstappen,1.0,0:03:27.071
1,Germany,1971-08-01,Jackie Stewart,12.0,1:29:15.700
2,Australia,1991-11-03,Ayrton Senna,14.0,0:24:34.899
3,Germany,1967-08-06,Denny Hulme,15.0,2:05:55.700
4,Pescara,1957-08-18,Stirling Moss,18.0,2:59:22.700
5,Germany,1951-07-29,Alberto Ascari,20.0,3:23:03.300
6,Germany,1957-08-04,Juan Manuel Fangio,22.0,3:30:38.300
7,Belgium,1958-06-15,Tony Brooks,24.0,1:37:06.300
8,Belgium,1970-06-07,Pedro Rodriguez,28.0,1:38:09.900
9,Spain,1975-04-27,Jochen Mass,29.0,0:42:53.700


In [21]:
# 2DA CONSULTA: Cuál es el país con mayor récord en la historia por número de vueltas 

#Hallando el valor máximo de la columna 'Laps'
valor_maximo = df_bd_python['Laps'].max()

#Seleccionando las filas por medio del metodo .loc aquellas que cumplan la condicion
df_con_2 = df_bd_python.loc[df_bd_python['Laps'] == valor_maximo].sort_values(by='Time',ascending=True).reset_index()

df_con_2.head(10)

,index,Grand Prix,Date,Winner,Laps,Time
0,102,Indianapolis 500,1960-05-30,Jim Rathmann,200.0,3:36:11.360
1,92,Indianapolis 500,1959-05-30,Rodger Ward,200.0,3:40:49.200
2,74,Indianapolis 500,1957-05-30,Sam Hanks,200.0,3:41:14.250
3,83,Indianapolis 500,1958-05-30,Jimmy Bryan,200.0,3:44:13.800
4,49,Indianapolis 500,1954-05-31,Bill Vukovich,200.0,3:49:17.270
5,32,Indianapolis 500,1952-05-30,Troy Ruttman,200.0,3:52:41.880
6,40,Indianapolis 500,1953-05-30,Bill Vukovich,200.0,3:53:01.690
7,66,Indianapolis 500,1956-05-30,Pat Flaherty,200.0,3:53:28.840
8,59,Indianapolis 500,1955-05-30,Bob Sweikert,200.0,3:53:59.530
9,8,Indianapolis 500,1951-05-30,Lee Wallard,200.0,3:57:38.050
